In [9]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from tqdm import tqdm

import amelia_datatools.utils.common as C

out_dir = f"{C.OUTPUT_DIR}/count"
os.makedirs(out_dir, exist_ok=True)


In [10]:
airport = "kdca"
base_dir = C.DATA_DIR
traj_version = C.VERSION

traj_dir = os.path.join(base_dir, f'traj_data_{traj_version}', 'raw_trajectories', f'{airport}')

traj_files = [os.path.join(traj_dir, f) for f in os.listdir(traj_dir)]
print(traj_files[0])

data = pd.read_csv(traj_files[0])
data.head()

/Users/alonso.cano/Developer/CMU/Amelia/AmeliaDataTools/datasets/amelia/traj_data_a10v08/raw_trajectories/kdca/KDCA_253_1681228800.csv


,Frame,ID,Altitude,Speed,Heading,Lat,Lon,Range,Bearing,Type,Interp,x,y
0,0,971520,12.5,11.533333,356.102051,38.855808,-77.039590,0.506485,-0.601733,0.0,[ORG],0.417524,-0.286707
1,0,971491,12.5,0.000000,120.541992,38.842980,-77.041540,1.105035,-2.716172,2.0,[ORG],-1.006538,-0.456053
2,0,971483,12.5,0.250000,89.103790,38.856114,-77.040368,0.573876,-0.665270,0.0,[ORG],0.451497,-0.354237
3,0,971515,12.5,0.400000,201.051636,38.842084,-77.042077,1.214878,-2.715001,2.0,[ORG],-1.106002,-0.502680
4,0,971141,12.5,0.000000,319.440674,38.842420,-77.041590,1.163656,-2.734820,1.0,[ORG],-1.068705,-0.460397


In [11]:
agent_counts = {
    'total': [], '0': [], '1': [], '2': []
}
total_count = 0
for f in tqdm(traj_files):
    data = pd.read_csv(f)

    unique_frames = np.unique(data.Frame)
    for frame in unique_frames:
        agents = data[data.Frame == frame]

        agent_counts['total'].append(agents.shape[0])
        agent_counts['0'].append(np.where(agents.Type == 0.0)[0].shape[0])
        agent_counts['1'].append(np.where(agents.Type == 1.0)[0].shape[0])
        agent_counts['2'].append(np.where(agents.Type == 2.0)[0].shape[0])


100%|██████████| 100/100 [01:26<00:00,  1.16it/s]


In [12]:
agent_count_stats = {}
for k, v in agent_counts.items():
    v = np.asarray(v)
    agent_count_stats[k] = {
        "min": round(v.min().astype(float), 5),
        "max": round(v.max().astype(float), 5),
        "mean": round(v.mean().astype(float), 5),
        "std": round(v.std().astype(float), 5)
    }

    agent_counts[k] = v

agent_count_stats

{'total': {'min': 1.0, 'max': 32.0, 'mean': 6.98787, 'std': 4.57956},
 '0': {'min': 0.0, 'max': 22.0, 'mean': 3.02973, 'std': 3.25195},
 '1': {'min': 0.0, 'max': 8.0, 'mean': 0.38718, 'std': 1.11216},
 '2': {'min': 0.0, 'max': 22.0, 'mean': 3.57097, 'std': 3.443}}

In [13]:
with open(f'{out_dir}/{airport}_stats.json', 'w') as f:
    json.dump(agent_count_stats, f, indent=2)

with open(f'{out_dir}/{airport}.pkl', 'wb') as handle:
    pickle.dump(agent_counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
name = {'total': 'Total', '0': 'Aircraft', '1': 'Vehicle', '2': 'Unknown'}
colors = {'ksea': 'blue', 'kewr': 'red', 'kbos': 'green', 'kmdw': 'orange', 'katl': 'purple'}
for key, value in name.items():
    arr = agent_counts[key]
    color = colors.get(airport, 'black')

    plt.hist(arr, bins = int(arr.max()), color=color)
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}.png", dpi=600)
    plt.close()

In [16]:
for key, value in name.items():
    arr = agent_counts[key]
    q_lower = np.quantile(arr, 0.05)
    q_upper = np.quantile(arr, 0.95)

    arr = arr[(arr >= q_lower) & (arr <= q_upper)]
    color = colors.get(airport, 'black')
    plt.hist(arr, bins = int(arr.max()), color=color)
    plt.title(f"{value}")
    plt.xlabel("Number of Agents per Timestep")
    plt.ylabel("Count")
    plt.savefig(f"{out_dir}/{airport}_{key}_iqr.png", dpi=600)
    plt.close()